In [4]:
import pandas as pd
import csv
from vzorci_in_pomozne_definicije import *

# Ustvarimo podatkovno bazo z merganjem dveh dataframov

In [5]:
osnovni_podatki_o_kartah = pd.read_csv(FILENAME_KARTE_CSV).replace(slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani)
dodatni_podatki_o_kartah = pd.read_csv(FILENAME_KARTE_DODATEK_CSV)
dodatni_podatki_o_setih = pd.read_csv(FILENAME_SETI_CSV).replace(slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani)

podatkovna_baza = pd.merge(osnovni_podatki_o_kartah, dodatni_podatki_o_kartah)
podatkovna_baza = pd.merge(podatkovna_baza, dodatni_podatki_o_setih)

%reset_selective -f osnovni_podatki_o_kartah
%reset_selective -f dodatni_podatki_o_kartah
%reset_selective -f dodatni_podatki_o_setih

Nato popravimo dataframe s funckijami, ki smo jih napisali. To je absoultno nujno, saj je na žalost veliko podatkov s spletne strani napačnih ali nekonsistentnih (to se na nekaterih straneh vidi celo vizualno).
Natančnejša razlaga in detajli se nahajajo v datoteki `vzorci_in_pomozne_definicije.py`

In [19]:
podatkovna_baza["cardtype"] = podatkovna_baza["cardtype"].apply(lambda x: popravi_cardtype_aftermath(x[1:-1].replace("'", "").split(', ')) if type(x) == str else x)
podatkovna_baza["subtype"] = podatkovna_baza["subtype"].apply(lambda x: popravi_subtype_adventure(x[1:-1].replace("'", "").split(', ')) if type(x) == str else x)
podatkovna_baza["supertype"] = podatkovna_baza["supertype"].apply(lambda x: x[1:-1].replace("'", "").split(', ') if type(x) == str else x)
podatkovna_baza["reserved_list"] = podatkovna_baza["reserved_list"].apply(lambda x: x == "Yes")
podatkovna_baza["barva"] = podatkovna_baza.apply(lambda x: popravi_barvo_karte(str(x.barva), str(x.oracle_text), str(x.cardtype)), axis = 1)

seznam_stolpcev_datumov = ["all_time_low_datum", "all_time_high_datum", "datum_izida"]
podatkovna_baza[seznam_stolpcev_datumov] = podatkovna_baza[seznam_stolpcev_datumov].apply(lambda x: pd.to_datetime(pretvori_datum_v_datetime(x)) if type(x) == str else x, axis=1)

seznam_stolpcev_cen = ["povprecna_cena", "povprecna_cena_foil", "povprecje_eu", "all_time_low", "all_time_high"]
for cena in seznam_stolpcev_cen:
    podatkovna_baza[cena] = podatkovna_baza[cena].apply(lambda x: (float(x[1:])) if (type(x) != float) else x).replace("nan","NaN")

podatkovna_baza.to_json(FILENAME_POPOLNI_PODATKI_KARTE_JSON, orient='records', indent=4)

# Nato
 ustvarimo povezovalne tabele

In [17]:
def ustvari_povezave(indeks, seznam, seznam_povezav):
    if type(seznam) != list:
        return (indeks, None)
    for element in seznam:
        seznam_povezav.append((indeks, element))

def ustvari_pare_povezovalne_tabele_glede_na_seznam(ime_id_stolpca, ime_stolpca_seznamov):
    seznam_povezav = []
    """
    To ustvari pare povezovalne tabele glede na podani imeni indeksnega stolpca in stolpca seznamov
    """
    podatkovna_baza.apply(lambda x: ustvari_povezave(x[ime_id_stolpca], x[ime_stolpca_seznamov], seznam_povezav), axis = 1)

    return(seznam_povezav)

In [18]:
for tip in ["cardtype","subtype","supertype"]:
    with open(os.path.join("Podatki", "Povezovalne_tabele", f"povezovalna_{tip}.csv"), "w", encoding="utf-8") as csvfile:
        seznam_tuplov = ustvari_pare_povezovalne_tabele_glede_na_seznam("id_karte",f"{tip}")
        writer = csv.writer(csvfile)
        writer.writerow(["id_karte", f"{tip}"])
        for vrstica in seznam_tuplov:
            writer.writerow(vrstica)